First I tried a simple 2 layer CNN model, consisting of two convolutional blocks (3→32, 32→64) each with ReLU and max pooling, flattenning the resulting 64×8×8 feature map into a 4096 dimensional vector, and applies a single dropout layer in the fully connected head. Its design makes it fast and lightweight, but without batch normalization or extra layers it can’t learn as deepand well regularized features as a deeper network would.So I moved on to a 3layer CNN. This three layer CNN adds a third convolutional block (with 3→32→64→128 filters), each including batch normalization, activation, pooling, and dropout, to progressively learn richer image features down to a 128×4×4 map. It then flattens these 2048 features into a 128 unit dense layer (with dropout) before the final three way classification.

In [1]:
#### 3 layer CNN:
import os
import pandas as pd
import numpy as np

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms


#Read training and test ssets
DATA_DIR = "data"
train_df = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
test_df  = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))
print("Train:", train_df.shape, "Test:", test_df.shape)

# Check for missing values
assert train_df.isna().sum().sum() == 0, "Missing in train!"
assert test_df .isna().sum().sum() == 0, "Missing in test!"


# Extract ids & labels
train_ids    = train_df.pop("id").values
## zero-based for PyTorch
train_labels = (train_df.pop("y").values - 1).astype(np.int64)  
test_ids     = test_df.pop("id").values

# Convert to float32 and reshape to (N,3,32,32)

X_train = train_df.values.astype(np.float32).reshape(-1, 3, 32, 32)
X_test  = test_df.values.astype(np.float32).reshape(-1, 3, 32, 32)

Train: (1200, 3074) Test: (1200, 3073)


In [6]:

# DataLoader
class FarmDataset(Dataset):
    def __init__(self, images, labels=None):
        self.images = torch.from_numpy(images)
        self.labels = None if labels is None else torch.from_numpy(labels)
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        x = self.images[idx]
        if self.labels is None:
            return x
        y = self.labels[idx]
        return x, y

# Train/validation split
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, train_labels,
    test_size=0.2,
    stratify=train_labels,
    random_state=42
)

batch_size   = 64
train_loader = DataLoader(FarmDataset(X_tr,  y_tr), batch_size, shuffle=True)
val_loader   = DataLoader(FarmDataset(X_val, y_val), batch_size)
test_loader  = DataLoader(FarmDataset(X_test),    batch_size)

# Define the 3-layer CNN
class ThreeLayerCNN(nn.Module):
    def __init__(self, num_classes=3):
        super().__init__()
        self.features = nn.Sequential(
            # Block 1: 3→32
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),            ### 32×32×32 → 32×16×16
            nn.Dropout(0.25),

            # Block 2: 32→64
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),              ##### 64×16×16 → 64×8×8
            nn.Dropout(0.25),

            # Block 3: 64→128
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),            #### 128×8×8 → 128×4×4
            nn.Dropout(0.25),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),                  ####→ 128*4*4 = 2048
            nn.Linear(2048, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# training setup
device    = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model     = ThreeLayerCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training and evaluation functions
def train_epoch(loader):
    model.train()
    total_loss = total_correct = 0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        preds  = model(xb)
        loss   = criterion(preds, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss   += loss.item() * xb.size(0)
        total_correct+= (preds.argmax(1) == yb).sum().item()
    return total_loss/len(loader.dataset), total_correct/len(loader.dataset)

def eval_epoch(loader):
    model.eval()
    total_loss = total_correct = 0
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            preds  = model(xb)
            total_loss   += criterion(preds, yb).item() * xb.size(0)
            total_correct+= (preds.argmax(1) == yb).sum().item()
    return total_loss/len(loader.dataset), total_correct/len(loader.dataset)

#  Train for 30 epochs, save  the best model
best_val_acc = 0.0
for epoch in range(1, 31):
    tr_loss, tr_acc = train_epoch(train_loader)
    vl_loss, vl_acc = eval_epoch(val_loader)
    if vl_acc > best_val_acc:
        best_val_acc = vl_acc
        torch.save(model.state_dict(), "best_three_layer_cnn.pth")
    print(f"Epoch {epoch:02d} | train_acc: {tr_acc:.3f} | val_acc: {vl_acc:.3f}")

print(f"\nBest validation accuracy: {best_val_acc*100:.2f}%")

# Reload best modeln and final validation
model.load_state_dict(torch.load("best_three_layer_cnn.pth"))
_, final_acc = eval_epoch(val_loader)
print(f"Reloaded best model val_acc: {final_acc*100:.2f}%")

# generate CSV
model.eval()
all_preds = []
with torch.no_grad():
    for xb in test_loader:
        xb    = xb.to(device)
        preds = model(xb).argmax(1).cpu().numpy() + 1  
        all_preds.append(preds)
all_preds = np.concatenate(all_preds)

submission = pd.DataFrame({
    "id": test_ids,
    "y":  all_preds
})
submission.to_csv("3L_cnn.csv", index=False)
print("Saved 3L_cnn.csv with", len(submission), "rows.")


Epoch 01 | train_acc: 0.669 | val_acc: 0.537
Epoch 02 | train_acc: 0.858 | val_acc: 0.887
Epoch 03 | train_acc: 0.925 | val_acc: 0.912
Epoch 04 | train_acc: 0.938 | val_acc: 0.958
Epoch 05 | train_acc: 0.930 | val_acc: 0.892
Epoch 06 | train_acc: 0.965 | val_acc: 0.933
Epoch 07 | train_acc: 0.963 | val_acc: 0.925
Epoch 08 | train_acc: 0.958 | val_acc: 0.963
Epoch 09 | train_acc: 0.969 | val_acc: 0.896
Epoch 10 | train_acc: 0.968 | val_acc: 0.921
Epoch 11 | train_acc: 0.970 | val_acc: 0.921
Epoch 12 | train_acc: 0.956 | val_acc: 0.825
Epoch 13 | train_acc: 0.960 | val_acc: 0.912
Epoch 14 | train_acc: 0.976 | val_acc: 0.963
Epoch 15 | train_acc: 0.974 | val_acc: 0.975
Epoch 16 | train_acc: 0.977 | val_acc: 0.917
Epoch 17 | train_acc: 0.977 | val_acc: 0.896
Epoch 18 | train_acc: 0.988 | val_acc: 0.933
Epoch 19 | train_acc: 0.967 | val_acc: 0.958
Epoch 20 | train_acc: 0.972 | val_acc: 0.942
Epoch 21 | train_acc: 0.957 | val_acc: 0.950
Epoch 22 | train_acc: 0.978 | val_acc: 0.954
Epoch 23 |